In [ ]:
import aif360
print("AIF360 is working!")


In [ ]:
import pandas as pd
import numpy as np

# AIF360 core components
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import ClassificationMetric
from aif360.algorithms.postprocessing import RejectOptionClassification


In [ ]:
import os
print(os.getcwd())




In [ ]:
df_1 = pd.read_csv("predictions_binary.csv")
df_1.head()


In [ ]:
df_2 = pd.read_csv("predictions_scores.csv")
df_2.head()

In [ ]:
score_columns = ["regression_beat","regression_grf_noZ","regression_grf_all",
     "regression_xgboost","regression_beat_log","regression_logistic"] 

#Making sure the probabilities reflect chance of the favourable outcome which is in our case non-recidivism
df_2[score_columns] = 1 - df_2[score_columns]
df_2.head()

In [ ]:
df_1[df_1['age_cat_Greater than 45'] == 1][['true_value', 'regression_logistic']].mean()

In [ ]:
df_1[df_1['sex_Male'] == 1][['true_value', 'regression_logistic']].mean()


In [ ]:
df_1[df_1['race_African-American'] == 1][['true_value', 'regression_logistic']].mean()


In [ ]:
prediction_cols = ["regression_beat", "regression_grf_all","regression_beat_log",
                   "regression_xgboost","regression_logistic","regression_grf_noZ"]

protected_info = {
    "race_African-American": {"unpriv": 1, "priv": 0},
    "sex_Male": {"unpriv": 1, "priv": 0 },
    "age_cat_Greater than 45" : {"unpriv": 0, "priv": 1}
    }



In [ ]:
def get_fairness_metric_table(df, prediction_cols, protected_info, true_label_col="true_value"):
    results = []

    for attr, group_vals in protected_info.items():
        unpriv_val = group_vals["unpriv"]
        priv_val = group_vals["priv"]

        for pred_col in prediction_cols:
            df_base = df.copy()

            # Create ground truth dataset
            gt_dataset = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label= 1,
                df=df_base,
                label_names=[true_label_col],
                protected_attribute_names=[attr]
            )

            # Replace label column with prediction
            df_base[true_label_col] = df_base[pred_col]

            # Create prediction dataset
            pred_dataset = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label= 1,
                df=df_base,
                label_names=[true_label_col],
                protected_attribute_names=[attr]
            )

            # Group mapping
            unprivileged_groups = [{attr: unpriv_val}]
            privileged_groups = [{attr: priv_val}]

            # Metrics
            metric = ClassificationMetric(
                gt_dataset,
                pred_dataset,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups
            )

            # Store results
            results.append({
                "protected_attribute": attr,
                "prediction_column": pred_col,
                 "version": "not-postprocessed",
                "accuracy": metric.accuracy(),
                "disparate_impact": metric.disparate_impact(),
                "equal_opportunity_diff": metric.equal_opportunity_difference(),
                "average_odds_diff": metric.average_odds_difference()
            })

    # Convert to DataFrame
    return pd.DataFrame(results)



In [ ]:
df_not = get_fairness_metric_table(df_1, prediction_cols, protected_info)
df_not.head(20)

In [ ]:
def get_roc_fairness_metrics_only(
    df,
    prediction_cols,
    protected_info,
    true_label_col="true_value",
    metric_name="Average odds difference",
    metric_lb=-0.05,
    metric_ub=0.05
):
    results = []

    for attr, group_vals in protected_info.items():
        unpriv_val = group_vals["unpriv"]
        priv_val = group_vals["priv"]

        for pred_col in prediction_cols:
            # Ground truth dataset
            gt_dataset = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label=1,
                df=df,
                label_names=[true_label_col],
                protected_attribute_names=[attr]
            )

            # Create dataset with raw scores (dummy label used)
            df_prob = df.copy()
            df_prob["true_value"] = 0

            pred_ds_probs = BinaryLabelDataset(
                favorable_label= 0,
                unfavorable_label= 1,
                df=df_prob,
                label_names=["true_value"],
                protected_attribute_names=[attr]
            )

            # Add scores
            pred_ds_probs.scores = df[pred_col].values.reshape(-1, 1)

            # Define group mapping
            unprivileged_groups = [{attr: unpriv_val}]
            privileged_groups = [{attr: priv_val}]

            # Apply ROC
            roc = RejectOptionClassification(
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups,
                low_class_thresh=0.01,
                high_class_thresh=0.99,
                num_class_thresh=100,
                num_ROC_margin=50,
                metric_name=metric_name,
                metric_ub=metric_ub,
                metric_lb=metric_lb
            )

            roc.fit(gt_dataset, pred_ds_probs)
            pred_ds_adj = roc.predict(pred_ds_probs)

            # Evaluate post-ROC fairness
            metric_adj = ClassificationMetric(
                gt_dataset,
                pred_ds_adj,
                unprivileged_groups=unprivileged_groups,
                privileged_groups=privileged_groups
            )

            results.append({
                "protected_attribute": attr,
                "prediction_column": pred_col,
                "version": "postprocessed",
                "accuracy": metric_adj.accuracy(),
                "disparate_impact": metric_adj.disparate_impact(),
                "equal_opportunity_diff": metric_adj.equal_opportunity_difference(),
                "average_odds_diff": metric_adj.average_odds_difference()
            })

    return pd.DataFrame(results)



In [ ]:
df_post = get_roc_fairness_metrics_only(df_2, prediction_cols, protected_info)
df_post.head(20)

In [ ]:
df_post.head(20)

In [ ]:
df_not.to_csv("notPost.csv", index=False)


In [ ]:
df_post.to_csv("Post.csv", index=False)
